### Recursive Implementation

In [2]:
# Basic, good for understanding, but slow for long strings due to exponential time complexity (not the best option)
# Time Complexity: O(3**n) (Exponential time complexity)

def levenshtein_recursive(a, b):
    if len(a) == 0:
        return len(b)
    if len(b) == 0:
        return len(a)
    
    if a[-1] == b[-1]:
        cost = 0
    else:
        cost = 1
    
    return min(levenshtein_recursive(a[:-1], b) + 1,
               levenshtein_recursive(a, b[:-1]) + 1,
               levenshtein_recursive(a[:-1], b[:-1]) + cost)

In [3]:
levenshtein_recursive('sunday', 'sand')

3

### Dynamic Programming Implementation

In [4]:
# Time complexitty: O(len1 × len2) in this case => O(6 × 4) = O(24) len(a) = 6 and len(b) = 4

def levenshtein_distance_dp(a, b):
    m, n = len(a), len(b)
    distance_matrix = [[0] * (n + 1) for k in range(m + 1)]    
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                distance_matrix[i][j] = j
            elif j == 0:
                distance_matrix[i][j] = i
            else:
                if a[i-1] == b[j-1]:
                    cost = 0
                else:
                    cost = 1
                distance_matrix[i][j] = min(distance_matrix[i-1][j] + 1,   
                                distance_matrix[i][j-1] + 1,      
                                distance_matrix[i-1][j-1] + cost) 
    
    return distance_matrix[m][n]

In [5]:
levenshtein_distance_dp('sunday', 'sand')

3

### Optimized Space Complexity (Using 1D Array)

In [6]:
# Time complexitty: O(len1 × len2) in this case => O(6 × 4) = O(24) len(a) = 6 and len(b) = 4
 
def levenshtein_optimized_sc(a, b):
    m, n = len(a), len(b)
    prev = [j for j in range(n + 1)]
    
    for i in range(1, m + 1):
        curr = [i] + [0] * n
        for j in range(1, n + 1):
            if a[i-1] == b[j-1]:
                cost = 0
            else:
                cost = 1
            curr[j] = min(prev[j] + 1,      # Deletion
                           curr[j-1] + 1,    # Insertion
                           prev[j-1] + cost) # Substitution
        prev = curr
    
    return prev[n]

In [7]:
levenshtein_optimized_sc('sunday', 'sand')

3

###  Using Levenshtein Libraries

In [8]:
import Levenshtein
print(Levenshtein.distance('sunday', 'sand'))

3


### Damerau-Levenshtein distance

In [ ]:
# Time complexitty: O(len1 × len2) in this case => O(6 × 4) = O(24) len(a) = 6 and len(b) = 4

def damerau_levenshtein_distance(a, b):
    len1, len2 = len(a), len(b)
    distance_matrix = [[0] * (len2 + 1) for k in range(len1 + 1)]

    for i in range(len1 + 1):
        distance_matrix[i][0] = i
    for j in range(len2 + 1):
        distance_matrix[0][j] = j

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            cost = 0 if a[i-1] == b[j-1] else 1
            distance_matrix[i][j] = min(
                distance_matrix[i-1][j] + 1,        
                distance_matrix[i][j-1] + 1,        
                distance_matrix[i-1][j-1] + cost    
            )

            if i > 1 and j > 1 and a[i-1] == b[j-2] and a[i-2] == b[j-1]:
                distance_matrix[i][j] = min(distance_matrix[i][j], distance_matrix[i-2][j-2] + cost)

    return distance_matrix[len1][len2]

In [10]:
damerau_levenshtein_distance('sunday', 'sand')

3

### Jaro Similarity

In [11]:
# Time complexitty: O(len1 × len2) in this case => O(6 × 4) = O(24) len(a) = 6 and len(b) = 4

def jaro_similarity(a, b):
    len1, len2 = len(a), len(b)
    
    if len1 == 0 and len2 == 0:
        return 1.0
    
    match_distance = (max(len1, len2) // 2) - 1
    
    matches1 = [0] * len1
    matches2 = [0] * len2
    
    matches = 0
    for i in range(len1):
        start = max(0, i - match_distance)
        end = min(len2, i + match_distance + 1)
        for j in range(start, end):
            if not matches2[j] and a[i] == b[j]:
                matches1[i] = matches2[j] = True
                matches += 1
                break
    
    if matches == 0:
        return 0.0
    
    transpositions = 0
    k = 0
    for i in range(len1):
        if matches1[i]:
            while not matches2[k]:
                k += 1
            if a[i] != b[k]:
                transpositions += 1
            k += 1
    
    jaro_sim = (
        (matches / len1) +
        (matches / len2) +
        ((matches - transpositions // 2) / matches)
    ) / 3
    
    return jaro_sim

In [12]:
jaro_similarity('sunday', 'sand')

0.75

### Jaro-Winkler Similarity

In [ ]:
# Time complexitty: O(len1 × len2) in this case => O(6 × 4) = O(24) len(a) = 6 and len(b) = 4

def jaro_winkler_similarity(a, b, precomputed_jaro=None):

    if precomputed_jaro is None:
        precomputed_jaro = jaro_similarity(a, b)  

    prefix_length = 0
    max_prefix = 4
    for i in range(min(len(a), len(b), max_prefix)):
        if a[i] == b[i]:
            prefix_length += 1
        else:
            break

    jaro_winkler = precomputed_jaro + (prefix_length * 0.1 * (1 - precomputed_jaro))

    return jaro_winkler

In [79]:
jaro_winkler_similarity('sunday', 'sand')

0.775

### Spell Checker (computing distances manually)

In [87]:
import nltk
from nltk.corpus import words
from concurrent.futures import ThreadPoolExecutor

nltk.download("words") # caching technique (optiization)


#function map (optimization)
distance_functions = {
    "levenshtein": levenshtein_distance_dp,
    "damerau_levenshtein": damerau_levenshtein_distance,
}
similarity_functions = {
    "jaro_similarity": jaro_similarity,
    "jaro_winkler": jaro_winkler_similarity,
}

def compute_distance(word, correct_word, method):
    if method in distance_functions:
        return correct_word, distance_functions[method](word, correct_word)
    similarity = similarity_functions[method](word, correct_word)
    return correct_word, 1 - similarity

def suggest_words(word, dictionary, method, max_suggestions=3, max_distance=2):
    word_len = len(word)
    precomputed_lengths = {w: len(w) for w in dictionary} #precompuation (optimization)
    
#parallel processing (optimization)
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(compute_distance, word, correct_word, method)
            for correct_word in dictionary
            if abs(word_len - precomputed_lengths[correct_word]) <= max_distance #early filtering (optimization)
        ]
        suggestions = [
            (correct_word, distance)
            for future in futures
            for correct_word, distance in [future.result()]
            if distance <= max_distance
        ]
    return [w for w, k in sorted(suggestions, key=lambda x: x[1])[:max_suggestions]] # sorting (optimization)

def main():
    dictionary = set(words.words())
    word = input("Enter a word to check: ").strip().lower()

    if word in dictionary:
        print(f"'{word}' is correct!")
        return
    for method in distance_functions | similarity_functions:
        print(f"\nUsing {method.replace('_', ' ').title()}: {suggest_words(word, dictionary, method)}")

[nltk_data] Downloading package words to C:\Users\Narmina
[nltk_data]     Mehtiyeva\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [88]:
main()

'set' is correct!


### Spell Checker (using libraries)

In [85]:
import nltk
from nltk.corpus import words
import jellyfish
from concurrent.futures import ThreadPoolExecutor  
import time

nltk.download("words") # caching technique (optiization)

# Precomputation (optimization)
DICTIONARY = set(words.words())
WORD_LENGTHS = {w: len(w) for w in DICTIONARY}

def jaro_winkler_distance(w1, w2): #  Function Map (optimization)
    return 1 - jellyfish.jaro_winkler_similarity(w1, w2)

DISTANCE_FUNCTIONS = {
    "levenshtein": jellyfish.levenshtein_distance,
    "damerau_levenshtein": jellyfish.damerau_levenshtein_distance,
    "jaro_similarity": lambda w1, w2: 1 - jellyfish.jaro_similarity(w1, w2),
    "jaro_winkler": jaro_winkler_distance,
}

def compute_distance(word, correct_word, metric):
    distance_func = DISTANCE_FUNCTIONS[metric]
    distance = distance_func(word, correct_word)
    return correct_word, distance

def spell_check(word, dictionary, metric="levenshtein", max_suggestions=3, max_distance=2):
    start_time = time.time()

    #Early Filtering + Precomputation (optimizations)
    word_len = len(word)
    first_letter = word[0]
    candidate_words = [
        w for w in dictionary
        if abs(WORD_LENGTHS[w] - word_len) <= max_distance and w[0] == first_letter
    ]
    
    suggestions = []
    with ThreadPoolExecutor() as executor:  # Parallel Processing (optimization)
        futures = [executor.submit(compute_distance, word, w, metric) for w in candidate_words]

        for future in futures:
            correct_word, distance = future.result()
            if distance <= max_distance:
                suggestions.append((correct_word, distance))

    # Sorting (optimization)
    suggestions.sort(key=lambda x: x[1])
    result = [w for w, k in suggestions[:max_suggestions]]

    end_time = time.time()
    print(f"Spell check using {metric} completed in {end_time - start_time:.4f} seconds")
    return result

def main():
    word = input("Enter a word to check: ").strip().lower()

    if word in DICTIONARY:
        print(f"'{word}' is correct!")
        return

    print(f"'{word}' not found. Suggestions:")

    for metric in ["levenshtein", "damerau_levenshtein", "jaro_similarity", "jaro_winkler"]:
        suggestions = spell_check(word, DICTIONARY, metric=metric)
        metric_name = metric.replace("_", " ").title()
        print(f"\nUsing {metric_name}:")
        if suggestions:
            print(f"Input: {word} → Suggestions: {', '.join(suggestions)}")
        else:
            print("No suggestions found.")

[nltk_data] Downloading package words to C:\Users\Narmina
[nltk_data]     Mehtiyeva\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [86]:
main()

'miilk' not found. Suggestions:
Spell check using levenshtein completed in 0.2960 seconds

Using Levenshtein:
Input: miilk → Suggestions: milk, mink, mail
Spell check using damerau_levenshtein completed in 0.1480 seconds

Using Damerau Levenshtein:
Input: miilk → Suggestions: milk, mink, mail
Spell check using jaro_similarity completed in 0.1920 seconds

Using Jaro Similarity:
Input: miilk → Suggestions: milk, milky, mil
Spell check using jaro_winkler completed in 0.1770 seconds

Using Jaro Winkler:
Input: miilk → Suggestions: milk, milky, mil
